<a href="https://colab.research.google.com/github/NanqiuH/CS4262-project/blob/main/Update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load training and testing datasets
train_csv_path = '/content/drive/MyDrive/Colab Notebooks/train.csv'
test_csv_path = '/content/drive/MyDrive/Colab Notebooks/test.csv'

# Load data
train_data = pd.read_csv(train_csv_path)
test_data = pd.read_csv(test_csv_path)

# Display the first few rows of the training and testing datasets
print("Training Data:")
print(train_data.head())

print("\nTesting Data:")
print(test_data.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Training Data:
   stock_id  date_id  seconds_in_bucket  imbalance_size  \
0         0        0                  0      3180602.69   
1         1        0                  0       166603.91   
2         2        0                  0       302879.87   
3         3        0                  0     11917682.27   
4         4        0                  0       447549.96   

   imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                        1         0.999812   13380276.64        NaN   
1                       -1         0.999896    1642214.25        NaN   
2                       -1         0.999561    1819368.03        NaN   
3                       -1         1.000171   18389745.62        NaN   
4                       -1         0.999532   17860614.95        NaN   

   near_price  bid_price  bid_size  ask_price   ask_size  wap    tar

In [3]:
# 1. Data Processing
def preprocess_missing_values_advanced(data):
    # Check numeric columns and calculate statistics
    numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
    categorical_cols = data.select_dtypes(include=['object', 'category']).columns

    # Fill numeric columns using median and group-wise median
    for col in numeric_cols:
        if data[col].isnull().sum() > 0:
            # Group by `stock_id` for more precise imputation
            if 'stock_id' in data.columns:
                data[col] = data.groupby('stock_id')[col].transform(
                    lambda x: x.fillna(x.median() if x.median() < np.percentile(x, 95) else x.mean())
                )
            # If still null, fill with overall median
            data[col] = data[col].fillna(data[col].median())

    # Fill categorical columns with mode
    for col in categorical_cols:
        data[col] = data[col].fillna(data[col].mode()[0])

    # Special handling for target variable (training data only)
    if 'target' in data.columns:
        data['target'] = data['target'].fillna(data['target'].median())

    # Check for remaining missing values
    if data.isnull().sum().sum() > 0:
        print("Warning: Missing values remain in the following columns:")
        print(data.isnull().sum()[data.isnull().sum() > 0])

    # Summary of processed data
    print("\nData after processing:")
    print(data.info())  # Summary of data types and non-null values
    print("\nSample rows of the processed data:")
    print(data.head())  # Display the first few rows of data for manual inspection

    return data

# Process training and testing datasets
print("\nProcessing training data...")
train_data = preprocess_missing_values_advanced(train_data)
print("\nProcessing testing data...")
test_data = preprocess_missing_values_advanced(test_data)

# Verify and print results
print("\nSummary after processing:")
print("Training Data Missing Values:\n", train_data.isnull().sum())
print("\nTesting Data Missing Values:\n", test_data.isnull().sum())

print("\nSample of processed training data:")
print(train_data.head())

print("\nSample of processed testing data:")
print(test_data.head())



Processing training data...

Data after processing:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5237980 entries, 0 to 5237979
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   stock_id                 int64  
 1   date_id                  int64  
 2   seconds_in_bucket        int64  
 3   imbalance_size           float64
 4   imbalance_buy_sell_flag  int64  
 5   reference_price          float64
 6   matched_size             float64
 7   far_price                float64
 8   near_price               float64
 9   bid_price                float64
 10  bid_size                 float64
 11  ask_price                float64
 12  ask_size                 float64
 13  wap                      float64
 14  target                   float64
 15  time_id                  int64  
 16  row_id                   object 
dtypes: float64(11), int64(5), object(1)
memory usage: 679.4+ MB
None

Sample rows of the processed data:
   sto

In [4]:

# 2. Feature Engineering
def add_features(data):
    # Add basic common features
    data['mid_price'] = (data['ask_price'] + data['bid_price']) / 2
    data['price_spread'] = (data['ask_price'] - data['bid_price']) / (data['mid_price'] + 1e-9)
    data['imbalance_ratio'] = data['imbalance_size'] / (data['matched_size'] + 1e-9)
    data['price_pressure'] = data['imbalance_size'] * (data['ask_price'] - data['bid_price'])
    data['imbalance_momentum'] = data['imbalance_size'].diff().fillna(0)

    # Rolling and grouped features
    def add_rolling_features(data, group_cols, target_col, windows):
        for window in windows:
            data[f'rolling_mean_{window}_{target_col}'] = data.groupby(group_cols)[target_col].transform(
                lambda x: x.rolling(window=window, min_periods=1).mean())
            data[f'rolling_std_{window}_{target_col}'] = data.groupby(group_cols)[target_col].transform(
                lambda x: x.rolling(window=window, min_periods=1).std())
        return data

    data = add_rolling_features(data, ['date_id', 'stock_id'], 'mid_price', [10, 20, 50])

    # Add top 10 correlated features
    data['scale_matched_size'] = data['matched_size'] / (
        data.groupby('stock_id')['matched_size'].transform('median'))
    data['rolling3_mean_imb1_auc_ask_size_auc_bid_size'] = data.groupby(['date_id', 'stock_id'])[
        'imbalance_size'].transform(lambda x: x.rolling(3, min_periods=1).mean())
    data['rolling18_mean_imb1_auc_ask_size_auc_bid_size'] = data.groupby(['date_id', 'stock_id'])[
        'imbalance_size'].transform(lambda x: x.rolling(18, min_periods=1).mean())
    data['rolling18_mean_imb1_ask_size_all_bid_size_all'] = data.groupby(['date_id', 'stock_id'])[
        'ask_size'].transform(lambda x: x.rolling(18, min_periods=1).mean())
    data['imb1_ask_money_bid_money'] = (data['ask_price'] * data['ask_size']) - (
        data['bid_price'] * data['bid_size'])
    data['rolling_ewm_12_imb1_reference_price_wap'] = data['reference_price'].ewm(
        span=12, adjust=False).mean()
    data['imb1_wap_mid_price'] = (data['wap'] - data['mid_price']) / (
        data['wap'] + data['mid_price'] + 1e-9)
    data['imb1_reference_price_bid_price'] = (data['reference_price'] - data['bid_price']) / (
        data['reference_price'] + data['bid_price'] + 1e-9)
    data['imb1_reference_price_ask_price'] = (data['reference_price'] - data['ask_price']) / (
        data['reference_price'] + data['ask_price'] + 1e-9)
    data['rolling18_mean_imb1_reference_price_wap'] = data['reference_price'].rolling(
        18, min_periods=1).mean()

    # Fill missing values after feature engineering
    data.fillna(0, inplace=True)

    return data

# Apply feature engineering to training and testing datasets
train_data = add_features(train_data)
test_data = add_features(test_data)

# Print results to confirm the new features have been added
print("Training Data with New Features:")
print(train_data.head())

print("\nTesting Data with New Features:")
print(test_data.head())

# Check for any remaining missing values
print("\nMissing Values After Feature Engineering (Train):")
print(train_data.isnull().sum())

print("\nMissing Values After Feature Engineering (Test):")
print(test_data.isnull().sum())


Training Data with New Features:
   stock_id  date_id  seconds_in_bucket  imbalance_size  \
0         0        0                  0      3180602.69   
1         1        0                  0       166603.91   
2         2        0                  0       302879.87   
3         3        0                  0     11917682.27   
4         4        0                  0       447549.96   

   imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                        1         0.999812   13380276.64   0.996826   
1                       -1         0.999896    1642214.25   0.999121   
2                       -1         0.999561    1819368.03   1.001351   
3                       -1         1.000171   18389745.62   0.998877   
4                       -1         0.999532   17860614.95   0.998720   

   near_price  bid_price  ...  scale_matched_size  \
0    0.997337   0.999812  ...            0.635182   
1    0.999929   0.999896  ...            0.593159   
2    1.000296   0.99